In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
events_1 = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:
# Convert to Delta
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",header=True ,inferSchema=True)
# Create managed table
df.write.format("delta").mode("overwrite").saveAsTable("events_table")

In [0]:
# SQL approach
spark.sql("DROP Table if exists  events_table")


In [0]:
spark.sql("""
        CREATE TABLE events_table
        USING DELTA 
        AS SELECT * FROM events_table
""")

In [0]:
# If table exist or not

spark.sql("SHOW TABLES").show()

In [0]:
spark.sql("SELECT * FROM events_delta").show(5)

In [0]:
# Test schema enforcement 
# Create wrong schema DataFrame


from pyspark.sql import Row 

wrong_schema_df = spark.createDataFrame([Row(id=2 , name="abc",value=1000)])

In [0]:
# Append it to the table

try:
    wrong_schema_df.write.format("delta").mode("append").saveAsTable("events_table")
except Exception as e:
    print(f"Enforcement schema: {e}")

# Read the table

#That's all

In [0]:
# Data Prep & Conversion to Delta
# Task: Convert CSV to Delta format. Concept: Writing to Delta format creates a _delta_log folder that tracks every transaction.

from pyspark.sql.functions import col, lit, concat_ws
from delta.tables import *

# 0. PREP: Re-combine our datasets (if not already in memory from Day 3)
df_full = events.withColumn("month", lit("Oct")) \
                .unionByName(events_1.withColumn("month", lit("Nov")))

# Define our storage path (Volumes is the preferred modern location)
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/delta/events_bronze"

# 1. WRITE TO DELTA
# mode("overwrite") replaces existing data safely (ACID transaction)
print(f"🚀 Writing {df_full.count():,} rows to Delta Lake...")

df_full.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_path)

print(f"✅ Data successfully written to: {delta_path}")

Creating Managed Tables (SQL & PySpark)
Task: Create Delta tables accessible via SQL. Concept: A "Managed Table" means Databricks manages both the metadata and the file storage.

In [0]:
# METHOD A: PySpark Approach (SaveAsTable)
# This registers the dataframe in the Hive Metastore
table_name = "workspace.ecommerce.events_silver"
df_full.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"✅ Managed Table created: {table_name}")

In [0]:
%sql

-- # METHOD B: SQL Approach (Querying the table we just created)
-- # Magic command allows us to switch languages comfortably
SELECT 
    brand, 
    count(*) as volume, 
    format_number(sum(price), 2) as total_sales
FROM workspace.ecommerce.events_silver
WHERE event_type = 'purchase'
GROUP BY brand
ORDER BY total_sales DESC
LIMIT 5;

Testing Schema Enforcement
Task: Test schema enforcement. Concept: Delta Lake is "schema-on-write." If you try to append data with the wrong columns, it throws an error to protect the table's integrity.

In [0]:

# 1. Create 'Bad' Data (Schema Mismatch)
# Our table expects: event_time, event_type, product_id...
# We provide: wrong_col_1, wrong_col_2
bad_data = [("error_test", 999)]
df_bad = spark.createDataFrame(bad_data, ["wrong_col_name", "wrong_value"])

print("🛡️ Attempting to write bad schema to Delta table...")

try:
    # Attempt to Append
    df_bad.write.format("delta").mode("append").save(delta_path)
    
except Exception as e:
    # We catch the error to demonstrate success
    print("\n✅ SUCCESS: Schema Enforcement blocked the write!")
    print(f"Error Message Snippet: {str(e)[:300]}...")

Handling Duplicates (The MERGE/Upsert)
Task: Handle duplicate inserts. Concept: In traditional data lakes, removing duplicates requires rewriting the whole file. In Delta, we use MERGE to perform an "Upsert" (Update if exists, Insert if new).

Note: Since this dataset lacks a unique Primary Key, we will simulate one using a composite key (User + Time + Product).

In [0]:
# 1. Simulate "New" Data that contains some duplicates
# We take 5 rows that already exist + 1 brand new row
existing_rows = df_full.limit(5)
new_row_data = [("2019-12-01 12:00:00", "purchase", 12345, 100, "test_code", "test_brand", 99.99, 88888,"92d96fde-8bb3-4e00-8c23-a032dfed738c", "Dec")]
new_row = spark.createDataFrame(new_row_data, df_full.columns)

# Combine to create our "incoming batch"
incoming_batch = existing_rows.union(new_row)

print(f"Incoming Batch Size: {incoming_batch.count()} (5 Duplicates + 1 New)")

# 2. Initialize Delta Table object
deltaTable = DeltaTable.forPath(spark, delta_path)

# 3. PERFORM MERGE (Upsert)
# Logic: Match on User + Time + Product. 
# If matched -> Ignore (don't insert duplicate). If not matched -> Insert.
deltaTable.alias("target").merge(
    incoming_batch.alias("source"),
    """
    target.user_id = source.user_id AND 
    target.event_time = source.event_time AND 
    target.product_id = source.product_id
    """
).whenNotMatchedInsertAll().execute()

print("✅ Merge Complete. Duplicates were ignored, new row was added.")

Verification & Visualization

In [0]:
from pyspark.sql.functions import col
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/delta/events_bronze"
# Verify the new row exists and we didn't duplicate the existing 5
display(
    spark.read.format("delta").load(delta_path)
    .filter(col("brand") == "test_brand")
)

🧠 Key Learnings & Takeaways
Performance: Writing to Delta format optimizes the file layout (Parquet + Transaction Log), making reads faster than raw CSV.
Safety: We proved that Schema Enforcement prevents accidental data corruption.
Upserts: We used the MERGE command to handle duplicates efficiently. This is standard in "Slowly Changing Dimension" (SCD) ETL pipelines.
Interoperability: We switched seamlessly between PySpark (saveAsTable) and SQL (SELECT) to interact with the same data.